In [1]:
import gurobipy as gb
import networkx as nx
from itertools import combinations, chain

#import pygraphviz as pygv


import os
from IPython.display import SVG, display

In [2]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

#
# Drawing 
# functions
#

#
# Drawing 
# functions
#

def DrawInitialGraph():
    global DrawG
    DrawG = pygv.AGraph(directed='true',strict='true', splines='true')


    for i in G.nodes():
        pos = str(G.node[i]['x'] * args.scale) + ',' + str((G.node[i]['y'])* args.scale)
        if i == root:
            DrawG.add_node (i, shape='circle', pos=pos, color="red", fontsize='8', width='0.3', fixedsize='true')
        else:
            DrawG.add_node (i, shape='circle', pos=pos, color="black", fontsize='8', width='0.3', fixedsize='true')   	

    DrawG.layout(prog='neato', args='-n')
    DrawG.draw (path=str(basename) + '.svg', format='svg')
    DrawG.clear()


def DrawSol (x):

    for i in G.nodes():
        pos = str(G.node[i]['x'] * args.scale) + ',' + str((G.node[i]['y'])* args.scale)
        DrawG.add_node (i, shape='circle', pos=pos,fontsize='8', width='0.3', fixedsize='true')

    DrawG.layout(prog='neato', args='-n')
    filepath=str(basename) + '_sol.svg'

    for i in G.edges():
        h = i[0]
        k = i[1]
        if x[i].x > 0.00001:
            lab = round(x[i].x,4)	
            if x[i].x > 0.999999:
                DrawG.add_edge(h, k, color='black', label=lab, fontsize='8')
            else:
                DrawG.add_edge(h, k, color='red', label=lab, fontsize='8')

    DrawG.draw (path=filepath, format='svg')
    DrawG.clear()

def DrawSubtour (x, subtour):
    for i in G.nodes():
        pos = str(G.node[i]['x'] * args.scale) + ',' + str((G.node[i]['y'])* args.scale)
        if i in subtour:
            DrawG.add_node (i, shape='circle', pos=pos, fontsize='8', width='0.3', fixedsize='true', style='filled')
        else:
            DrawG.add_node (i, shape='circle', pos=pos, fontsize='8', width='0.3', fixedsize='true')

    DrawG.layout(prog='neato', args='-n')
    filepath=str(basename) + '_sub.svg'

    for i in G.edges():
        h = i[0]
        k = i[1]
        if x[i].x > 0.00001:
            lab = round(x[i].x,4)	
            if x[i].x > 0.999999:
                DrawG.add_edge(h, k, color='black', label=lab, fontsize='8')
            else:
                DrawG.add_edge(h, k, color='red', label=lab, fontsize='8')

    DrawG.draw (path=filepath, format='svg')
    DrawG.clear()

    

In [3]:
class args:
    filename = None
    scale = 20

In [4]:
# 
# Read the graph in the graphML format
#

args.filename = 'net29052018R2.gml'

basename = os.path.splitext(args.filename)[0]

G = nx.read_graphml (args.filename, node_type=int)

print ("G has", G.number_of_nodes(), "nodes and", G.number_of_edges(), "edges")

D = G.to_directed()

G has 30 nodes and 435 edges


In [7]:
#G.nodes()
#G.edges()

#D.nodes()
#D.edges()

In [9]:
root = 1

In [ ]:
DrawInitialGraph()

display(SVG(filename=basename+'.svg'))

In [10]:
pcst = gb.Model()


#
# Variables definition
#


x = pcst.addVars(D.edges(), vtype=gb.GRB.BINARY, \
                 obj = [- D[i][j]['cost'] for i,j in D.edges()], \
                 name = 'x')

y = pcst.addVars (G.nodes(), vtype = gb.GRB.BINARY, \
                  obj = [G.node[i]['revenue'] for i in G.nodes()],\
                 name = 'y')

#cst = pcst.addVars (G.nodes(), vtype = gb.GRB.CONTINUOUS, \
#                  obj = [G.node[i]['customers'] for i in G.nodes()],\
#                 name = 'cst')

u = pcst.addVars (G.nodes(), vtype = gb.GRB.CONTINUOUS, lb = 0.0, ub = G.number_of_nodes(),\
                 name = 'u')


pcst.ModelSense = gb.GRB.MAXIMIZE

#
# Root is in the solution
#

pcst.addConstr(y[root] == 1, name = 'Fix')

pcst.update()

#
# Each node has exactly one incoming arc
#

pcst.addConstrs((x.sum('*',j) == y[j] \
                 for j in G.nodes() if j != root), \
                name='Node')

pcst.update()

#
# Precedence constraints (MTZ constraints)
#

pcst.addConstrs(((G.number_of_nodes() +  1) * x[i,j] \
                + u[i] - u[j] <= G.number_of_nodes() \
                 for i,j in D.edges()),\
                name = 'MTZ')

pcst.update()

#
# Connectivity constraints
#

pcst.addConstrs((x[j,k] <= y[j] for j in G.nodes() \
                 if j != root for k in D.neighbors(j)), name = 'Connect')
pcst.update()

In [11]:
#
# Each node may accept up to three connections
#

pcst.addConstrs((x.sum('*',j) <= 3 \
                 for j in G.nodes() if j != root), \
                name='ThreeNode')

pcst.update()

pcst.write('exercise#1.lp')

In [12]:
pcst.optimize()

Optimize a model with 1770 rows, 930 columns and 6004 nonzeros
Variable types: 30 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+02, 5e+03]
  Bounds range     [1e+00, 3e+01]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 618.0000000
Presolve removed 88 rows and 31 columns
Presolve time: 0.01s
Presolved: 1682 rows, 899 columns, 4988 nonzeros
Variable types: 29 continuous, 870 integer (870 binary)

Root relaxation: objective 8.558323e+03, 60 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 8558.32258    0   20  618.00000 8558.32258  1285%     -    0s
H    0     0                    1446.0000000 8558.32258   492%     -    0s
     0     0 7316.51613    0   32 1446.00000 7316.51613   406%     -    0s
H    0     0                    3586.0000000 7316.51613   10

In [13]:
sol = {}

for i,j in D.edges():
    if x[i,j].x > 0:
        sol[i,j] = x[i,j].x

In [14]:
sol

{(1, 15): 1.0,
 (2, 16): 1.0,
 (7, 14): 1.0,
 (8, 17): 1.0,
 (9, 10): 1.0,
 (9, 24): 1.0,
 (10, 25): 1.0,
 (11, 27): 1.0,
 (14, 4): 1.0,
 (15, 20): 1.0,
 (16, 21): 1.0,
 (17, 28): 1.0,
 (19, 23): 1.0,
 (20, 7): 1.0,
 (20, 26): 1.0,
 (21, 11): 1.0,
 (22, 9): 1.0,
 (23, 2): 1.0,
 (26, 8): 1.0,
 (27, 6): 1.0,
 (27, 22): 1.0,
 (28, 19): 1.0}

In [ ]:
DrawSol(x)
display(SVG(filename=basename+'_exercise#1_sol.svg'))

In [15]:
pcst.objVal

6183.0

In [16]:
D = G.to_directed()

root = 1

root_in_star = list(D.in_edges(root))

D.remove_edges_from(root_in_star)

In [41]:
DrawInitialGraph()

display(SVG(filename=basename+'.svg'))

G has 30 nodes and 435 edges


In [17]:
H = 4

In [18]:
pcst = gb.Model()

#
# Variables definition
#


x = pcst.addVars(D.edges(), vtype=gb.GRB.BINARY, \
                 obj = [- D[i][j]['cost'] \
                        for (i,j) in D.edges()], \
                 name = 'x')

y = pcst.addVars (G.nodes(), vtype = gb.GRB.BINARY, \
                  obj = [G.node[i]['revenue'] for i in G.nodes()],\
                 name = 'y')

#cst = pcst.addVars (G.nodes(), vtype = gb.GRB.CONTINUOUS, \
#                  obj = [G.node[i]['customers'] for i in G.nodes()],\
#                 name = 'cst')


Gnodesminusroot = list(G.nodes())

Gnodesminusroot.remove(root)


yl = pcst.addVars (Gnodesminusroot, range(1, H + 1), \
                   vtype = gb.GRB.BINARY,\
                   name = 'yl')


pcst.ModelSense = gb.GRB.MAXIMIZE

In [19]:
#
# Root is in the solution
#

pcst.addConstr(y[root] == 1, name = 'Root')

pcst.update()

In [20]:
#
# Each node has exactly one incoming arc
#

pcst.addConstrs((x.sum('*',j) == y[j] for j in G.nodes() \
                 if j != root), name='Indegr')

pcst.update()

In [21]:
#
# GSEC of size two
#

pcst.addConstrs((x[i,j] + x [j,i] <= y[j] for i in G.nodes() 
                 for j in G.nodes() if D.has_edge(i,j) and 
                 D.has_edge(j,i)), \
                name='GSEC')


pcst.update()

In [22]:
#
# Node-Hop Link
#

pcst.addConstrs((yl.sum(j,'*') == y[j] for j in G.nodes() \
                if j != root), name = 'NHLink')


pcst.update()

In [23]:
#
# Root Link
#

pcst.addConstrs((x[i,j] == yl[j,1] \
                 for i,j in D.out_edges(root)),\
              name = 'RootLink')


pcst.update()

pcst.write('exercise#1_nodelayer.lp')

In [24]:
#
# H-Link Initial
#

pcst.addConstrs((yl[i,h-1] + x[i,j] <= 1 + yl[j,h] \
                 for i,j in D.edges() for h in range(2,H+1) \
                 if i != root), name='HLinkI')


pcst.update()

pcst.write('exercise#1_nodelayer.lp')

In [25]:
#
# H-Link final
#

pcst.addConstrs((yl[i,H] + x[i,j] <= 1 for i,j in D.edges() if i != root),\
                name = 'HLinkE')
                
pcst.update()

pcst.write('exercise#1_nodelayer.lp')

In [26]:
pcst.optimize()

Optimize a model with 4148 rows, 987 columns and 12442 nonzeros
Variable types: 0 continuous, 987 integer (987 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 618.0000000
Presolve removed 30 rows and 30 columns
Presolve time: 0.04s
Presolved: 4118 rows, 957 columns, 13195 nonzeros
Variable types: 0 continuous, 957 integer (957 binary)

Root relaxation: objective 6.663811e+03, 131 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 6663.81111    0   86  618.00000 6663.81111   978%     -    0s
H    0     0                    1938.0000000 6663.81111   244%     -    0s
H    0     0                    2852.0000000 6663.81111   134%     -    0s
H    0     0                    3281.0000000 6663.81111   1

In [ ]:
DrawSol(x)
display(SVG(filename=basename+'_exercise#1_sol.svg'))

In [27]:
pcst.objVal

4221.0

In [28]:
D = G.to_directed()

In [29]:
pcst = gb.Model()

#
# Variables definition
#


x = pcst.addVars(D.edges(), vtype=gb.GRB.BINARY, \
                 obj = [- D[i][j]['cost'] for i,j in D.edges()], \
                 name = 'x')

y = pcst.addVars (G.nodes(), vtype = gb.GRB.BINARY, \
                  obj = [G.node[i]['revenue'] for i in G.nodes()],\
                 name = 'y')

#cst = pcst.addVars (G.nodes(), vtype = gb.GRB.CONTINUOUS, \
#                  obj = [G.node[i]['customers'] for i in G.nodes()],\
#                 name = 'cst')

u = pcst.addVars (G.nodes(), vtype = gb.GRB.CONTINUOUS, lb = 0.0, ub = G.number_of_nodes(),\
                 name = 'u')


pcst.ModelSense = gb.GRB.MAXIMIZE

In [30]:
#
# Root is in the solution
#

pcst.addConstr(y[root] == 1, name = 'Fix')

pcst.update()

In [31]:
#
# Each node has exactly one incoming arc
#

pcst.addConstrs((x.sum('*',j) == y[j] \
                 for j in G.nodes() if j != root), \
                name='Node')

pcst.update()

In [32]:
#
# Precedence constraints (MTZ constraints)
#

pcst.addConstrs(((G.number_of_nodes() +  1) * x[i,j] \
                + u[i] - u[j] <= G.number_of_nodes() \
                 for i,j in D.edges()),\
                name = 'MTZ')

pcst.update()

In [33]:
#
# Connectivity constraints
#

pcst.addConstrs((x[j,k] <= y[j] for j in G.nodes() \
                 if j != root for k in D.neighbors(j)), name = 'Connect')
pcst.update() 


In [34]:
pcst.write('exercise#1_perc.lp')

In [35]:
pcst.optimize()

Optimize a model with 1741 rows, 930 columns and 5163 nonzeros
Variable types: 30 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+02, 5e+03]
  Bounds range     [1e+00, 3e+01]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 618.0000000
Presolve removed 59 rows and 31 columns
Presolve time: 0.01s
Presolved: 1682 rows, 899 columns, 4988 nonzeros
Variable types: 29 continuous, 870 integer (870 binary)

Root relaxation: objective 8.558323e+03, 60 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 8558.32258    0   20  618.00000 8558.32258  1285%     -    0s
H    0     0                    1446.0000000 8558.32258   492%     -    0s
     0     0 7316.51613    0   32 1446.00000 7316.51613   406%     -    0s
H    0     0                    3586.0000000 7316.51613   10

In [36]:
sol = {}

for i,j in D.edges():
    sol[i,j] = x[i,j].x
sol
#sum(sol.values())

{(1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (1, 5): 0.0,
 (1, 6): 0.0,
 (1, 7): -0.0,
 (1, 8): 0.0,
 (1, 9): 0.0,
 (1, 10): 0.0,
 (1, 11): 0.0,
 (1, 12): 0.0,
 (1, 13): 0.0,
 (1, 14): -0.0,
 (1, 15): 1.0,
 (1, 16): 0.0,
 (1, 17): 0.0,
 (1, 18): 0.0,
 (1, 19): 0.0,
 (1, 20): -0.0,
 (1, 21): 0.0,
 (1, 22): 0.0,
 (1, 23): 0.0,
 (1, 24): 0.0,
 (1, 25): 0.0,
 (1, 26): -0.0,
 (1, 27): 0.0,
 (1, 28): 0.0,
 (1, 29): 0.0,
 (1, 30): 0.0,
 (2, 1): 0.0,
 (2, 3): -0.0,
 (2, 4): 0.0,
 (2, 5): 0.0,
 (2, 6): 0.0,
 (2, 7): 0.0,
 (2, 8): 0.0,
 (2, 9): 0.0,
 (2, 10): 0.0,
 (2, 11): 0.0,
 (2, 12): 0.0,
 (2, 13): 0.0,
 (2, 14): 0.0,
 (2, 15): 0.0,
 (2, 16): 1.0,
 (2, 17): 0.0,
 (2, 18): 0.0,
 (2, 19): -0.0,
 (2, 20): 0.0,
 (2, 21): -0.0,
 (2, 22): 0.0,
 (2, 23): -0.0,
 (2, 24): 0.0,
 (2, 25): 0.0,
 (2, 26): 0.0,
 (2, 27): 0.0,
 (2, 28): 0.0,
 (2, 29): 0.0,
 (2, 30): 0.0,
 (3, 1): 0.0,
 (3, 2): -0.0,
 (3, 4): 0.0,
 (3, 5): 0.0,
 (3, 6): 0.0,
 (3, 7): 0.0,
 (3, 8): 0.0,
 (3, 9): 0.0,
 (3, 10): 0.0,
 (3, 11): 

In [39]:
percList = {(i): G.node[i]['customers'] for i in G.nodes()}
percList

{1: 1830,
 2: 2099,
 3: 2827,
 4: 1447,
 5: 2592,
 6: 1133,
 7: 2385,
 8: 2347,
 9: 1560,
 10: 2955,
 11: 2573,
 12: 1060,
 13: 1325,
 14: 2552,
 15: 2211,
 16: 1381,
 17: 1094,
 18: 1513,
 19: 2597,
 20: 1057,
 21: 2193,
 22: 1865,
 23: 1840,
 24: 1116,
 25: 1427,
 26: 1456,
 27: 2710,
 28: 2293,
 29: 1095,
 30: 2149}

In [41]:
sum(percList.values())

56682

In [43]:
perc = 85 * sum(percList.values()) / 100
perc

48179.7

In [44]:
pcst.addConstr(y.prod(percList) >= perc, 'Perc')

pcst.update() 

pcst.write('exercise#1_perc.lp')

In [45]:
pcst.optimize()

Optimize a model with 1742 rows, 930 columns and 5193 nonzeros
Variable types: 30 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+02, 5e+03]
  Bounds range     [1e+00, 3e+01]
  RHS range        [1e+00, 5e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint Perc by 4058.700000000

Presolve removed 59 rows and 31 columns
Presolve time: 0.01s
Presolved: 1683 rows, 899 columns, 5017 nonzeros
Variable types: 29 continuous, 870 integer (870 binary)
Found heuristic solution: objective -35248.00000

Root relaxation: objective 8.558323e+03, 60 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 8558.32258    0   20 -35248.000 8558.32258   124%     -    0s
H    0     0                    -3774.000000 8558.32258   327%     -    0s
H    0     0                

In [ ]:
DrawSol(x)
display(SVG(filename=basename+'_exercise#1_sol.svg'))

In [46]:
pcst.objVal

6115.0

### After designing different networks, we can assume:
###     1.network - 6183.0
###     2.network - 4221.0
###     3.network - 6115.0
### the second one is the most profitable implementation